In [1]:
# ! pip install fastparquet

In [6]:
import pandas as pd
import swifter
import numpy as np
from glob import glob
import json
import gc


In [7]:
train = pd.read_parquet("../data_phase1/train.parquet")
val = pd.read_parquet("../data_phase1/validation.parquet")


In [8]:
train[["context_type"]] = train[["context_type"]].fillna(value="NA")
val[["context_type"]] = val[["context_type"]].fillna(value="NA")


In [12]:
pd.options.display.max_rows=None
pd.options.display.max_columns=None


In [9]:
attribute_df = pd.read_parquet("../data_phase1/attributes.parquet")

In [5]:
max_date = attribute_df["start_online_date"].max()
min_date = attribute_df["start_online_date"].min()


In [10]:
attribute_df["start_online_date"] = (attribute_df["start_online_date"] - min_date) / (max_date - min_date)


In [13]:
attribute_df.head(2)

,product_id,gender,main_colour,second_colour,season,collection,category_id_l1,category_id_l2,category_id_l3,brand_id,season_year,start_online_date,material_values,attribute_values
0,0013f07ccdf212210c110e63f0de46e37669c17a4d855a...,a8c9cca4c116691f1e331a5058e84f05e31696bc4f611c...,7673fc4fdc325f3785a223787d2b32e381e8b4c1c8a765...,4737cd35940c2338e96c18a25aeb6848d46f0da795bce8...,847a067597e39838f1f85b0774f44e68b4d6e64d3ec4dd...,a3791e8d85c005b0d9d60d6d3b7e8edd2f256a5cc928d0...,abf367e49718254e068eee51a565fdcede2d741e4c7e33...,61fe255948ec07c4eb25c70f7144b54beddd00466ad866...,05f872d8b8ec85642ad49786d6e443c0df6e7df4bdcba3...,dd6ea8954a945ef0889f30d57b7fdb8d6aaad397e6c6ff...,c7c4ac6af030e54d02b9e4545e4223e76515c3ce4e498e...,0.331994,f61ecea9b45f1590e57706b88207449bdd4cb703b917ad...,8b45c5d5e010acf257787f2ce0c505857d94709c436991...
1,002239cd57f19f22e557030dff363dfbd1344d8f7ac829...,4a00d8b84bdb2ec2f219304d3883a46336f9fb38d2f1e6...,0f97dafafa5dc4bb18853ea00776dfcc52302f40411b50...,ca8c396e7422e324d4454a911e0319d07b85a4fb89b006...,9db35d237f5873f0410d3ca18c07430270086eb1e7838d...,a3791e8d85c005b0d9d60d6d3b7e8edd2f256a5cc928d0...,e54f8513b708db3afdbd4950bd3420579a8cddabf4c1b3...,3809cade495cd7dc289e6aee521d380549ebd3456f03bc...,fd021cd2dbaf0d7b6105a1b136cf5a094e025010a2096f...,a6536c6bc250d525ccd3b63a3ec483a33a2010422932a3...,ef7d8d2e02aec8c328dafde95486f7181c37d07be3d167...,0.066907,1675f293342bbb518ba3a5ad39399aa0a13580653d253e...,ca8c396e7422e324d4454a911e0319d07b85a4fb89b006...


In [6]:
train = train.merge(attribute_df, on='product_id', how='left')
val = val.merge(attribute_df, on='product_id', how='left')


In [7]:
train.head(2)

,query_id,user_id,session_id,product_id,page_type,previous_page_type,device_category,device_platform,user_tier,user_country,...,season,collection,category_id_l1,category_id_l2,category_id_l3,brand_id,season_year,start_online_date,material_values,attribute_values
0,92d4dd491a874a2cf92c8d311a44a42b597c64a5ede23d...,e5e4c71b1b9456dafece1338762d4ee3db698cf32c384c...,2740b0d77b4e6fafd75321f7d0794210afa8bd650955e7...,bf056e3841dd3a358c6aacb1f9e74e4c7c4adc62e33b45...,596618814963e496d74434df8b8fe3306892f2e4ce6aaa...,c9f34437ce0e536fefd11a34b9a411b541d2dabfec872a...,d6538f13ace825448d0af4fa5e58d2d08fa2d0850e5e14...,d6538f13ace825448d0af4fa5e58d2d08fa2d0850e5e14...,d74a5cebc23c56af60a5768c22d44b52f598629d4011fa...,fec032cb05435471f2305006f4a1ba994c9d2f4bcad8ef...,...,9db35d237f5873f0410d3ca18c07430270086eb1e7838d...,4c8006c7e513057a9138641abc2a9e65f4b014a8045259...,642878c18e40a1028b6610cd3396d4ee4fb6ad9623f5bb...,a37fdccd28723fba01a2f0a68d3bb1a3b847e550a7b0c2...,ca8c396e7422e324d4454a911e0319d07b85a4fb89b006...,206b88b91a241cc6ed5af856ce3ddda4dccf91ae2ec798...,dc1194ba428d5cd4c49f8a769a6577ac1042162da38bc1...,570.041667,f61ecea9b45f1590e57706b88207449bdd4cb703b917ad...,28d2b1e7b0970e2c58966c47b4c7ab7426e43a8689007e...
1,541a93bd95c3f4127a53e6b0d4b41db55ad9cb9e19d34a...,fca847f7eb5a5a21991421354b0f26afb4a517e540541f...,d5feab37634fd140e85b8f98dcb909a8779b4f0417c73a...,c6513ec49c8e04c265c907933799ff76f24c075c6308c4...,06a7f8e972f61aeb0e06335699518079a444e4450ff766...,d74a5cebc23c56af60a5768c22d44b52f598629d4011fa...,d6538f13ace825448d0af4fa5e58d2d08fa2d0850e5e14...,d6538f13ace825448d0af4fa5e58d2d08fa2d0850e5e14...,d179859aac8f7c1f88e1ee29b6655596873318c55127d3...,8c82855f15d05cd74fa59956434df17522fc68e4ce3900...,...,847a067597e39838f1f85b0774f44e68b4d6e64d3ec4dd...,4c8006c7e513057a9138641abc2a9e65f4b014a8045259...,f6bafdefb3c04b392adc1d139ea55a565c8829627bb7d3...,884c59e2814e352d318a8447c2f32a0370955af64838e3...,393b930ad3c40c6af1c14ce8572f39a3a2027ef70dd21e...,de60eee3c057eab83a9987275b3ecfb080026c3297b036...,ef7d8d2e02aec8c328dafde95486f7181c37d07be3d167...,208.041667,c4aebf95aa0044c5230c104faa9ef2b5ccfab836b764d4...,ca8c396e7422e324d4454a911e0319d07b85a4fb89b006...


In [8]:
def reduce_mem_usage(df):
    start_mem = df.memory_usage().sum() / 1024 ** 2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024 ** 2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df


In [9]:
train = reduce_mem_usage(train)
val = reduce_mem_usage(val)


Memory usage of dataframe is 802.92 MB
Memory usage after optimization is: 274.46 MB
Decreased by 65.8%
Memory usage of dataframe is 152.04 MB
Memory usage after optimization is: 67.06 MB
Decreased by 55.9%


In [10]:
gc.collect()

22

In [11]:
train.shape, val.shape

((3507990, 29), (687192, 28))

In [12]:
train.memory_usage().sum()/(1024*1024*1024), val.memory_usage().sum()/(1024*1024*1024)

(0.2680312283337116, 0.06548863649368286)

In [13]:
train.context_type.unique()

['designer_id', 'product_id', 'designer_id,category_id', 'NA', 'category_id']
Categories (5, object): ['designer_id', 'product_id', 'designer_id,category_id', 'NA', 'category_id']

In [14]:
def similarity_util(df1, df2):
    res = []

    if df1['gender'].values[0] == df2['gender'].values[0]:
        res.append(1)
    else:
        res.append(0)
        
    if df1['main_colour'].values[0] == df2['main_colour'].values[0]:
        res.append(1)
    else:
        res.append(0)    
    
    if df1['second_colour'].values[0] == df2['second_colour'].values[0]:
        res.append(1)
    else:
        res.append(0)   
        
    if df1['season'].values[0] == df2['season'].values[0]:
        res.append(1)
    else:
        res.append(0)          
        
    if df1['collection'].values[0] == df2['collection'].values[0]:
        res.append(1)
    else:
        res.append(0)          
        
    if df1['category_id_l1'].values[0] == df2['category_id_l1'].values[0]:
        res.append(1)
    else:
        res.append(0)           
        
    if df1['category_id_l2'].values[0] == df2['category_id_l2'].values[0]:
        res.append(1)
    else:
        res.append(0)       
    
    if df1['category_id_l3'].values[0] == df2['category_id_l3'].values[0]:
        res.append(1)
    else:
        res.append(0)          
    
    
    if df1['brand_id'].values[0] == df2['brand_id'].values[0]:
        res.append(1)
    else:
        res.append(0)     
    
    if df1['season_year'].values[0] == df2['season_year'].values[0]:
        res.append(1)
    else:
        res.append(0)      
    
    a_mat = df1['material_values'].values[0].split(',')
    b_mat = df2['material_values'].values[0].split(',')
    commmon_length_mat = len(set(a_mat).intersection(set(b_mat)))
    if commmon_length_mat>0:
        res.append(commmon_length_mat/max(len(a_mat), len(b_mat)))
    else:
        res.append(0)
        
    a_attr = df1['attribute_values'].values[0].split(',')
    b_attr = df2['attribute_values'].values[0].split(',')
    common_length_attr = len(set(a_attr).intersection(set(b_attr)))
    if common_length_attr>0:
        res.append(common_length_attr/max(len(a_attr), len(b_attr)))
    else:
        res.append(0)
        
    
    res.append(1.0 - abs(df1['start_online_date'].values[0] - df2['start_online_date'].values[0]))
    
    return res


In [15]:
def jacquard_similarity(row, df):
    product_id_1 = row["product_id"]
    context_type = row["context_type"]
    
    if context_type in ["product_id", "designer_id"]:
        product_id_2 = row["context_value"]
        df1 = df[df['product_id']==product_id_1]
        df2 = df[df['product_id']==product_id_2]
        if len(df2)>0:
            res = similarity_util(df1, df2)
            return sum(res)/len(res)
    elif context_type=="designer_id,category_id":
        product_id_2 = row["context_value"].split(",")[0]
        df1 = df[df['product_id']==product_id_1]
        df2 = df[df['product_id']==product_id_2]
        if len(df2)>0:
            res = similarity_util(df1, df2)
            return sum(res)/len(res)
    else:
        return None
        
    
    

In [ ]:
train["product_context_similarity"] = train.swifter.apply(lambda x: jacquard_similarity(x, attribute_df), 1)


In [ ]:
val["product_context_similarity"] = val.swifter.apply(lambda x: jacquard_similarity(x, attribute_df), 1)


In [ ]:
del attribute_df
gc.collect()


In [ ]:
# train.to_csv("train_product_context_sim.csv", index=False)

In [ ]:
# val.to_csv("val_product_context_sim.csv", index=False)